# Training a Convolutional Neural Network for a computer vision binary classification task

From https://github.com/dsikar/city-dss-cnn-workshop-1


In [ ]:
# get the data
# Install PyDrive
!pip install PyDrive

#Import modules
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Get the shareable link e.g. https://drive.google.com/file/d/19sU5tk8RQY605GGjPJMW8mx3S9HrKaFW/view?usp=sharing
# Get the id from the link 1JYdxKasFKNfVqx2Iklj7iv_bwPanmi4j
downloaded = drive.CreateFile({'id':"19sU5tk8RQY605GGjPJMW8mx3S9HrKaFW"})   
# Remember at least what was the file extension
downloaded.GetContentFile('datasets.tar.gz')       

In [ ]:
# list drive contents
!ls -lh

total 531M
-rw-r--r-- 1 root root 2.7K Nov  5 17:13 adc.json
-rw-r--r-- 1 root root 266M Nov  5 17:14 datasets.tar.gz
-rw-r--r-- 1 root root 266M Nov  5 17:13 fire_data.tar.gz
drwxr-xr-x 1 root root 4.0K Oct 28 16:30 sample_data


In [ ]:
# unpack compressed files
!tar xvf datasets.tar.gz

In [1]:
# list again to verity
!ls

sample_data


In [ ]:
DATADIR = 'datasets/'
CATEGORIES = ['fire', 'nofire']

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [ ]:
IMG_SIZE = 64
def create_training_data():
    training_data = []
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category) 
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=C 1=O

        for img in tqdm(os.listdir(path)):  # iterate over each image
            try:
                img_array = cv2.imread(os.path.join(path,img))  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
              
    return training_data

In [ ]:
training_data = create_training_data()

In [ ]:
import random

print(len(training_data))
random.shuffle(training_data)
for sample in training_data[:10]:
    print(sample[1])

In [ ]:
X = []
Y = []

for features,label in training_data:
    X.append(features)
    Y.append(label)

# try to get around split error
Y = np.asarray(Y)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
X = X/255.0
X.shape[1:]

In [ ]:
type(Y)

In [ ]:
# # set up image augmentation
# from keras.preprocessing.image import ImageDataGenerator

# datagen = ImageDataGenerator(
#     rotation_range=15,
#     horizontal_flip=True,
#     width_shift_range=0.1,
#     height_shift_range=0.1
#     #zoom_range=0.3
#     )
# datagen.fit(X)

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D

model = Sequential()


model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=X.shape[1:]))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D())
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(units=256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=128, activation='relu'))

model.add(Dense(units=2, activation = 'softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()



In [ ]:
history = model.fit(X, Y, batch_size=32, epochs=100,validation_split=0.3)
# model.fit_generator(datagen.flow(X, Y, batch_size=32),
#                     epochs=100,
#                     verbose=1)

#history = model.fit(X, Y, batch_size=27, epochs=10,validation_split=0.3)

In [ ]:
print(history.history.keys())

In [ ]:
model.save('firemodel.h5')

In [ ]:
!ls -lh

In [ ]:
from matplotlib import pyplot as plt
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# Using the model in practice
import tensorflow as tf
from tensorflow.keras.utils import plot_model
# model = tf.keras.models.load_model('TrainedModels/Fire-64x64-color-v7-soft.h5')
model = tf.keras.models.load_model('firemodel.h5')

# TODO
# single prediction fire/no fire

# TODO, download model and run locally
